## Setup

In [1]:

%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma beautifulsoup4
!pip install -q langchain_google_genai

In [1]:
import os
from dotenv import load_dotenv
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "RAG_With_Memory"
os.environ["LANGSMITH_API_KEY"] = "Use your API key"

In [20]:

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
["not found "if GOOGLE_API_KEY == "" else "found"][0]

'found'

import warnings
warnings.filterwarnings('ignore')

In [22]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", api_key=GOOGLE_API_KEY)
gemini_embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x78591e904150>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.0-pro",convert_system_message_to_human=True)

In [24]:

import bs4
from langchain import hub

In [25]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
doc = loader.load()


In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)


In [29]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x785929edaa90>, search_kwargs={})

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [35]:
question_answering_chain=create_stuff_documents_chain(model, chat_prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)
response = rag_chain.invoke({"input":"what is MRKL?"})


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [36]:
response["answer"]

'MRKL is an abbreviation for "Modular Reasoning, Knowledge, and Language." It is a neuro-symbolic architecture for autonomous agents. \nA MRKL system contains a collection of "expert" modules and a general-purpose LLM that routes inquiries to the best suitable expert module.'

In [37]:
from langchain.chains import create_history_aware_retriever

retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)


contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [38]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


from langchain_core.messages import HumanMessage, AIMessage


chat_history = []


question1 = "what is Task Decomposition?"

In [39]:

message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})
message1["answer"]

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Task Decomposition is a technique used to break down complex tasks into smaller, more manageable steps. It involves identifying the subgoals and steps required to achieve the main task. Task Decomposition can be done using different methods, such as prompting an LLM with simple instructions, using task-specific instructions, or with human inputs.'

In [40]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)
chat_history

[HumanMessage(content='what is Task Decomposition?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Task Decomposition is a technique used to break down complex tasks into smaller, more manageable steps. It involves identifying the subgoals and steps required to achieve the main task. Task Decomposition can be done using different methods, such as prompting an LLM with simple instructions, using task-specific instructions, or with human inputs.', additional_kwargs={}, response_metadata={})]

In [41]:
second_question = "What are common ways of doing it?"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Common ways of doing Task Decomposition include:

1. **Using an LLM (Large Language Model)**: Prompting an LLM with simple instructions like "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?" can help decompose the task into smaller steps.

2. **Using Task-Specific Instructions**: If available, task-specific instructions can provide guidance on how to break down the task. For example, for writing a novel, one might use an outline template as a guide.

3. **With Human Inputs**: Involving humans in the Task Decomposition process can ensure that the decomposition aligns with the desired outcome and considers domain-specific knowledge.


In [42]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Task decomposition is a technique used to break down a complex task into smaller, more manageable steps. \nThis can be done using simple prompting, task-specific instructions, or human inputs. \nTask decomposition can help improve model performance on complex tasks by making them more manageable and easier to solve.'

In [43]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is a technique used to break down a complex task into smaller, more manageable steps. \nThis can be done using simple prompting, task-specific instructions, or human inputs. \nTask decomposition can help improve model performance on complex tasks by making them more manageable and easier to solve.', additional_kwargs={}, response_metadata={})])}

In [44]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is a technique used to break down a complex task into smaller, more manageable steps. 
This can be done using simple prompting, task-specific instructions, or human inputs. 
Task decomposition can help improve model performance on complex tasks by making them more manageable and easier to solve.

